In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

#### Data preprocessing

In [3]:
df = pd.read_csv('anime.csv')
df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [4]:
df['genre'] = df['genre'].fillna("Unknown")

In [5]:
df['type'] = df['type'].fillna('Unknown')

In [6]:
df['rating'] = df['rating'].fillna(df['rating'].mean())

In [7]:
df['episodes'] = pd.to_numeric(df['episodes'], errors='coerce')
df['episodes'] = df['episodes'].fillna(0).astype(int)

In [8]:
df['name'] = df['name'].str.lower()
df['genre'] = df['genre'].str.lower()
df['type'] = df['type'].str.lower()

In [9]:
tfidf =TfidfVectorizer(stop_words='english')

In [10]:
Tfidf_matrix = tfidf.fit_transform(df['genre'])

In [11]:
print("TF-IDF matrix shape:", Tfidf_matrix.shape)

TF-IDF matrix shape: (12294, 47)


In [12]:
final_df = df[['anime_id', 'name', 'genre', 'type', 'episodes', 'rating', 'members']]
print(final_df.head())

   anime_id                              name  \
0     32281                    kimi no na wa.   
1      5114  fullmetal alchemist: brotherhood   
2     28977                          gintama°   
3      9253                       steins;gate   
4      9969                     gintama&#039;   

                                               genre   type  episodes  rating  \
0               drama, romance, school, supernatural  movie         1    9.37   
1  action, adventure, drama, fantasy, magic, mili...     tv        64    9.26   
2  action, comedy, historical, parody, samurai, s...     tv        51    9.25   
3                                   sci-fi, thriller     tv        24    9.17   
4  action, comedy, historical, parody, samurai, s...     tv        51    9.16   

   members  
0   200630  
1   793665  
2   114262  
3   673572  
4   151266  


#### Feature extraction

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(df['genre'])
print("TF-IDF shape:", tfidf_matrix.shape)

TF-IDF shape: (12294, 47)


This creates a sparse matrix where each genre word is a feature.

In [16]:
from sklearn.preprocessing import MinMaxScaler

In [17]:
# Select numeric features
numeric_features = df[['rating', 'episodes']].fillna(0)

In [18]:
# Normalize values to [0,1]
scaler = MinMaxScaler()
numeric_scaled = scaler.fit_transform(numeric_features)

In [20]:
from scipy.sparse import hstack

In [21]:
feature_matrix = hstack([tfidf_matrix, numeric_scaled])
print("Final feature matrix shape:", feature_matrix.shape)

Final feature matrix shape: (12294, 49)


We have one feature matrix that represents each anime with both genre info and numeric info (rating, episodes).
This will be used in the next step for cosine similarity & recommendations.

#### Recommendation System

In [22]:
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
cosine_sim = cosine_similarity(feature_matrix, feature_matrix)

In [24]:
indices = pd.Series(df.index,index=df['name']).drop_duplicates()

In [25]:
def recommend_anime(title, num_recommendations=5, threshold=0.5):
    if title not in indices:
        return "Anime not found in dataset!"
    
    idx = indices[title]
    
    # Get similarity scores for the chosen anime
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort by similarity score (highest first)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Apply threshold (ignore low similarity)
    sim_scores = [s for s in sim_scores if s[1] >= threshold]
    
    # Pick top N (excluding itself at index 0)
    sim_scores = sim_scores[1:num_recommendations+1]
    
    anime_indices = [i[0] for i in sim_scores]
    
    return df[['name','genre','rating']].iloc[anime_indices]

In [27]:
print(recommend_anime("kimi no na wa.", num_recommendations=5, threshold=0.4))

                                       name  \
5805            wind: a breath of heart ova   
6394           wind: a breath of heart (tv)   
1111  aura: maryuuin kouga saigo no tatakai   
878           shakugan no shana ii (second)   
1201         angel beats!: another epilogue   

                                                  genre  rating  
5805               drama, romance, school, supernatural    6.35  
6394               drama, romance, school, supernatural    6.14  
1111       comedy, drama, romance, school, supernatural    7.67  
878   action, drama, fantasy, romance, school, super...    7.79  
1201                        drama, school, supernatural    7.63  


#### Evaluation

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

For evaluation, we need to decide when a recommendation is “relevant.”
Common approach in content-based recommenders:

An anime is relevant if it belongs to the same genre or has a similar rating range as the target anime.

Otherwise, it’s “not relevant.”

###### Metrics
We use Precision, Recall, F1-score.

Precision = Relevant recommended anime / Total recommended anime
→ How many of the recommended were actually relevant.

Recall = Relevant recommended anime / Total relevant anime in dataset
→ How many of the possible relevant anime we managed to recommend.

F1-score = Harmonic mean of precision & recall.
→ Balances both.

In [30]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [31]:
def evaluate_recommender(test_df, num_recommendations=5, threshold=0.4):
    y_true = []
    y_pred = []

    for title in test_df['name'][:50]:  # limit to 50 for speed
        try:
            recs = recommend_anime(title, num_recommendations, threshold)
            target_genre = df[df['name'] == title]['genre'].values[0]

            # mark recommendations as relevant if genres overlap
            for rec in recs['name']:
                rec_genre = df[df['name'] == rec]['genre'].values[0]
                relevant = any(g in rec_genre for g in target_genre.split(", "))
                y_true.append(1 if relevant else 0)
                y_pred.append(1)  # system always "recommends"
        except:
            continue

    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return precision, recall, f1

precision, recall, f1 = evaluate_recommender(test_df)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 1.0
Recall: 1.0
F1-score: 1.0


###### Analysis

If precision is high but recall is low → recommender is too strict (few but accurate results).

If recall is high but precision is low → recommender is too broad (many results, not always relevant).

F1 helps balance both.

#### 1. Can you explain the difference between user-based and item-based collaborative filtering?

##### 🔹 User-Based Collaborative Filtering (UBCF)
Idea: Find users who are similar to the target user, and recommend items that those similar users liked.

How:

1.Compare users based on their item ratings/history.

2.Identify “neighbors” (users with similar tastes).

3.Recommend items liked by these neighbors but not yet consumed by the target user.

Example:
If User A and User B both liked Naruto and Bleach, and User B also liked One Piece, then recommend One Piece to User A.

Pros: Personalizes strongly for each user.

Cons:

Struggles when user base is large (scalability).

Suffers from the cold-start problem for new users (no history yet).

##### 🔹 Item-Based Collaborative Filtering (IBCF)
Idea: Find items that are similar to the item the user has already liked, and recommend them.

How:

1.Compare items based on ratings from users.

2.Build an item–item similarity matrix.

3.Recommend items most similar to what the user has liked/rated highly.

Example:
If many users who liked Attack on Titan also liked Tokyo Ghoul, then Tokyo Ghoul will be recommended to anyone who liked Attack on Titan.

Pros:

More stable, since item similarities don’t change as often as user preferences.

Works better with large datasets.

Cons:

Still suffers from the item cold-start problem (new item with no ratings can’t be recommended).

##### 🔹 Key Difference

User-based CF focuses on user similarity (find like-minded users).

Item-based CF focuses on item similarity (find similar items).

#### 2. What is collaborative filtering, and how does it work?

###### 🔹 What is Collaborative Filtering?

Collaborative Filtering (CF) is a recommendation technique that makes predictions about a user’s interests by collecting preferences (ratings, likes, interactions) from many users.

The basic assumption is:
👉 “If two users liked similar items in the past, they will probably like similar items in the future.”

It doesn’t need detailed information about the items (like genre, description, etc.) — it relies only on user–item interaction data.


###### 🔹 How Collaborative Filtering Works

There are two main approaches:

1. User-Based Collaborative Filtering

Step 1: Find users who are similar to the target user (neighbors).

Step 2: Recommend items that those similar users liked but the target user hasn’t consumed yet.

Example: If you and I have both rated many of the same anime highly, and you liked Steins;Gate, the system will recommend Steins;Gate to me.

2. Item-Based Collaborative Filtering

Step 1: Find items that are similar to each other (based on ratings from users).

Step 2: Recommend items similar to those the user already liked.

Example: If many people who liked Naruto also liked Bleach, then Bleach will be recommended to someone who liked Naruto.

In [ ]:
###### 🔹 Strengths

Very effective when there is a lot of user–item interaction data.

Learns directly from user behavior (no need for manual tagging of items).

###### 🔹 Limitations

Cold Start Problem: Can’t recommend well for new users (no history) or new items (no ratings).

Data Sparsity: If most users rate only a few items, it’s hard to find meaningful overlaps.

Scalability: With millions of users/items, computing similarities can be expensive.